In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#From https://www.kaggle.com/rohanrao/ashrae-half-and-half

from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False,verbose=True):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    if verbose :print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    if verbose:print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
%%time
 
train_df = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
train_df = reduce_mem_usage(train_df)
train_df.reset_index(inplace=True,drop=True)
train_df.to_feather('train.feather')
del train_df

example_test = pd.read_csv('/kaggle/input/jane-street-market-prediction/example_test.csv')
example_test = reduce_mem_usage(example_test)
example_test.reset_index(inplace=True,drop=True)
example_test.to_feather('example_test.feather')

features = pd.read_csv('/kaggle/input/jane-street-market-prediction/features.csv')
features = reduce_mem_usage(features)
features.reset_index(inplace=True,drop=True)
features.to_feather('features.feather')

example_sample_submission = pd.read_csv('/kaggle/input/jane-street-market-prediction/example_sample_submission.csv')
example_sample_submission = reduce_mem_usage(example_sample_submission)
example_sample_submission.reset_index(inplace=True,drop=True)
example_sample_submission.to_feather('example_sample_submission.feather')